<a href="https://colab.research.google.com/github/fppfurtado/ranqueador-cadernos-tec/blob/main/Ranqueador_de_T%C3%B3picos_TEC_Concursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Entrada de Dados
nome_arquivo = "b119290c-7edc-43a2-a347-e73070e51853.xlsx"  # @param {type: "string"}
total_questoes = "20"  # @param {type: "string"}
percentual_alvo = "80"  # @param {type: "string", placeholder: "entre com o número percental (ex.: 70, 80, etc)"}

print(nome_arquivo)
print(total_questoes)
print(percentual_alvo)

b119290c-7edc-43a2-a347-e73070e51853.xlsx
20
80


# Ranqueador

## Imports

In [ ]:
!pip install anytree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from anytree import Node, RenderTree
from copy import deepcopy
from google.colab import drive

## Funções

In [ ]:
def carregar_dados(planilha):
    """Carrega os dados da planilha e filtra linhas válidas"""
    df = pd.read_excel(planilha)
    df = df[df['Hierarquia'].notna() | df['Índice'].notna()]  # Remove linhas vazias
    return df

In [ ]:
def corrige_hierarquia_ausente(df, inicio_sequencia):
    df_corrigido = df.copy()

    filtro_linhas_sem_hierarquia = df['Hierarquia'].isna() & df['Índice'].notna()
    df_linhas_sem_hierarquia = df[filtro_linhas_sem_hierarquia]
    df_linhas_nova_hierarquia = df_linhas_sem_hierarquia

    sequencia_hierarquias_faltantes = list(range(inicio_sequencia, inicio_sequencia + len(df_linhas_sem_hierarquia)))
    df_linhas_nova_hierarquia["Hierarquia"] = [str(hierarquia).zfill(2) for hierarquia in sequencia_hierarquias_faltantes]
    indices_linhas_hierarquia_inserida = df_linhas_nova_hierarquia.index.tolist()

    df_corrigido["Hierarquia"] = df_corrigido["Hierarquia"].apply(lambda x: x if not pd.isna(x) else "")

    for i in range(len(indices_linhas_hierarquia_inserida)):
        for i_row, row in df_corrigido.iterrows():
            if indices_linhas_hierarquia_inserida[i] != indices_linhas_hierarquia_inserida[-1]:
                if indices_linhas_hierarquia_inserida[i] <= i_row < indices_linhas_hierarquia_inserida[i+1]:
                    if row["Hierarquia"]:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"] + "." + str(row["Hierarquia"])
                    else:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"]
                else:
                    break
            else:
                if i_row >= indices_linhas_hierarquia_inserida[i]:
                    if row["Hierarquia"]:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"] + "." + str(row["Hierarquia"])
                    else:
                        df_corrigido.loc[i_row, "Hierarquia"] = df_linhas_nova_hierarquia.loc[indices_linhas_hierarquia_inserida[i], "Hierarquia"]

    return df_corrigido

In [ ]:
def inserir_caminho(raiz, caminho):
    """
    Insere uma sequência de nós em uma árvore a partir do nó raiz.
    Cria apenas os nós que ainda não existem no caminho.

    :param raiz: Nó raiz da árvore (objeto Node)
    :param caminho: Lista com os nomes dos nós no caminho
    :return: O último nó do caminho
    """
    no_atual = raiz

    for nome in caminho:
        # Verifica se o nó já existe entre os filhos
        filhos = {filho.name: filho for filho in no_atual.children}
        if nome in filhos:
            no_atual = filhos[nome]
        else:
            novo_no = Node(nome, parent=no_atual)
            no_atual = novo_no  # Atualiza para o novo nó

    return no_atual

In [ ]:
def construir_arvore(df, valor_raiz="00", topico_raiz=None, perc_raiz=100):
    """Constrói a árvore hierárquica a partir do DataFrame"""
    raiz = Node(valor_raiz, topico=topico_raiz, porcentagem=perc_raiz)

    df_arvore = df.copy()
    df_arvore["Nível"] = df["Hierarquia"].apply(lambda x: len(str(x).split("."))+1  if not pd.isna(x) else 1)
    df_arvore["Caminho"] = df["Hierarquia"].apply(lambda x: str(x).split("."))

    for _, row in df_arvore.iterrows():
      node = inserir_caminho(raiz, row["Caminho"])
      node.topico = row["Índice"]
      node.porcentagem = float(row["Porcentagem"].rstrip("%"))

    return raiz

In [ ]:
def ranquear_topicos(raiz):
  raiz_copia = Node(raiz.name, topico=raiz.topico, porcentagem=raiz.porcentagem)

  if raiz.children:
    ordenados = sorted(raiz.children, key=lambda x: x.porcentagem, reverse=True)
    for filho in ordenados:
      filho_ordenado = ranquear_topicos(filho)
      filho_ordenado.parent = raiz_copia

  return raiz_copia

In [ ]:
def simular_prova(raiz, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido=None):
  raiz_copia = Node(raiz.name, topico=raiz.topico, porcentagem=raiz.porcentagem)

  if raiz.children:
    acumulado = 0
    alvo_relativo = (float(percentual_alvo) / 100) * raiz.porcentagem

    for filho in raiz.children:
      if acumulado >= alvo_relativo:
        break

      if topico_removido and topico_removido.lower() in filho.topico.lower():
        continue

      filho_ordenado = simular_prova(filho, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido=topico_removido)
      filho_ordenado.parent = raiz_copia
      acumulado += filho_ordenado.porcentagem

  return raiz_copia

In [ ]:
def recuperar_hierarquia(node):
  if node.parent:
    return recuperar_hierarquia(node.parent) + '.' + node.name
  else:
    return node.name

## Main

In [ ]:
df = carregar_dados(f"/content/drive/MyDrive/PROJETOS/CONCURSOS/{nome_arquivo}")
df_corrigido = corrige_hierarquia_ausente(df, inicio_sequencia=1)

arvore = construir_arvore(df_corrigido, topico_raiz="Direito Tributário")
arvore_ranqueada = ranquear_topicos(arvore)
arvore_simulada = simular_prova(arvore_ranqueada, total_questoes=total_questoes, percentual_alvo=percentual_alvo, topico_removido="Mesclad")

# Mostrando a árvore
for pre, fill, node in RenderTree(arvore_simulada):
  porcentagem_relativa = round(node.porcentagem / node.parent.porcentagem * 100, 2) if node.parent else ''
  texto_porcentagem = str(node.porcentagem) + '%' + (' / ' + str(porcentagem_relativa) + '%' if porcentagem_relativa else '')
  print(f"{pre}{node.name} - {node.topico} ({texto_porcentagem})")

print(f"Total de Tópicos: {len(arvore_simulada.descendants) + 1}")

00 - Direito Tributário (100%)
└── 01 - Direito Tributário (84.02% / 84.02%)
    ├── 07 - Crédito Tributário (arts. 139 a 193 do CTN) (15.98% / 19.02%)
    │   ├── 04 - Extinção do Crédito Tributário (arts. 156 a 174 do CTN) (4.61% / 28.85%)
    │   ├── 02 - Lançamento e Constituição do Crédito Tributário (arts. 142 a 150 do CTN) (3.94% / 24.66%)
    │   ├── 03 - Suspensão da Exigibilidade do Crédito Tributário (arts. 151 a 155-A do CTN) (2.22% / 13.89%)
    │   ├── 05 - Exclusão do Crédito Tributário (arts. 175 a 182 do CTN) (1.89% / 11.83%)
    │   └── 07 - Garantias e Privilégios do Crédito Tributário (arts. 183 a 193 do CTN) (1.05% / 6.57%)
    ├── 06 - Obrigação Tributária (arts. 113 a 138 do CTN) (15.21% / 18.1%)
    │   ├── 07 - Responsabilidade Tributária (arts. 128 a 138 do CTN) (4.5% / 29.59%)
    │   ├── 01 - Disposições Gerais sobre Obrigação Tributária (Conceito, Obrigação Principal e Acessória) (3.33% / 21.89%)
    │   ├── 02 - Fato Gerador (arts. 114 a 118 do CTN) (2.0% 

<ipython-input-5-b4f3962d7648>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_linhas_nova_hierarquia["Hierarquia"] = [str(hierarquia).zfill(2) for hierarquia in sequencia_hierarquias_faltantes]


In [ ]:
drive.mount('/content/drive')
DIRETORIO_NOTEBOOK = "/content/drive/MyDrive/PROJETOS/CONCURSOS/"

# Salvar árvore em arquivo .txt
with open(f"{DIRETORIO_NOTEBOOK}/arvore - {arvore_simulada.topico.lower()}.txt", 'w') as f:
    for pre, _, node in RenderTree(arvore_simulada):
        f.write(f"{pre}{node.name} - {node.topico} ({node.porcentagem}%)\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
